# XYT mining op de weg

In deze project identificeren we start- en stoplocaties op basis van XYT gegevens. 

In [1]:
import pandas as pd

#df["minutes"].iloc[762:771] = df["minutes"].iloc[762:771] + 1440 geeft warning: setting with copy --> weet niet wat het probleem is, werkt prima
pd.set_option('mode.chained_assignment', None)

from math import radians, cos, sin, asin, sqrt


In [2]:
#inlezen csv
df = pd.read_csv("XYT_data.csv", sep=";")

Nu is het tijd om het verschil in meters te berekenen tussen twee XY. De afstand berekenen we met behulp van de Haversine-formule.  

In [3]:
#In de Haversine-formule berekenen we de afstand tussen punt a en b. Om dit te berekenen zijn creëren we de kolommen lon_a / lat_a/ lon_b / lat_b. 
df["Lon_a"] = df["Lon"]
df["Lat_a"] = df["Lat"]
df["Lon_b"] = df["Lon"].shift(-1)
df["Lat_b"] = df["Lat"].shift(-1)

In [4]:
## Daarna creëren we de functie haversine ((https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points).

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

## Creërt kolom "distance". Deze kolom geeft verschil aan tussen punt a en b.
for index, row in df.iterrows():
    df.loc[index, 'distance'] = haversine(row['Lon_a'], row['Lat_a'], row['Lon_b'], row['Lat_b'])

## Verschil afstand in meters
df["distance"] = df["distance"].multiply(1000)

Eerst berekenen we het verschil in tijd tussen twee XY punten. Dit doen we door de tijdgegevens op te splitsen in datum en tijd. Daarna rekenen we de tijd om naar minuten. Als laatste berekenen we het verschil in minuten tussen twee XY punten. 

In [7]:
#Naar leesbare datumvermelding 
df["date_time"] = pd.to_datetime(df["Dt"])

#verschillende kolommen voor tijd en datum
df["date"] = df["date_time"].dt.date
df["time"] = df["date_time"].dt.time

#Tijd naar minuten omrekenen
minutes = pd.DatetimeIndex(df['date_time'])
df["minutes_a"] = minutes.hour * 60 + minutes.minute

#deel van de tijd op volgende dag --> minuten gaan van 1439 naar 0 --> maar gezien we in de code de minuten van elkaar moeten aftrekken + 1440 gedaan
df["minutes_a"].iloc[762:771] = df["minutes_a"].iloc[762:771] + 1440
df["minutes_b"] = df["minutes_a"].shift(-1).fillna(0).astype(int)

#Verschil in tijd afstanden (km/u en m/s)
df["dif_min"] = df["minutes_b"] - df["minutes_a"]
df["dif_sec"] = df["dif_min"].multiply(60)
df["speed_ms"] = df["distance"]/df["dif_sec"]
df["speed_kmu"] = df["distance"].divide(1000)/df["dif_sec"].divide(60*60)

In [8]:
# verwijderen van kolommen + volgorde veranderen

df_old = df

df = df_old[["VgNr", "Dt", "date_time", "date", "time", "GPS Coordinaten", "Lon_a", "Lat_a", "Lon_b", "Lat_b", "distance", "minutes_a", "minutes_b", "dif_min", "dif_sec", "speed_ms", "speed_kmu"]]

df[0:10].head

<bound method NDFrame.head of      VgNr                        Dt           date_time        date      time  \
0  378299  2017-09-28T11:18:25.000Z 2017-09-28 11:18:25  2017-09-28  11:18:25   
1  378300  2017-09-28T11:19:25.000Z 2017-09-28 11:19:25  2017-09-28  11:19:25   
2  378301  2017-09-28T11:20:25.000Z 2017-09-28 11:20:25  2017-09-28  11:20:25   
3  378302  2017-09-28T11:21:25.000Z 2017-09-28 11:21:25  2017-09-28  11:21:25   
4  378303  2017-09-28T11:22:25.000Z 2017-09-28 11:22:25  2017-09-28  11:22:25   
5  378304  2017-09-28T11:23:25.000Z 2017-09-28 11:23:25  2017-09-28  11:23:25   
6  378305  2017-09-28T11:24:25.000Z 2017-09-28 11:24:25  2017-09-28  11:24:25   
7  378306  2017-09-28T11:25:25.000Z 2017-09-28 11:25:25  2017-09-28  11:25:25   
8  378307  2017-09-28T11:26:25.000Z 2017-09-28 11:26:25  2017-09-28  11:26:25   
9  378308  2017-09-28T11:27:25.000Z 2017-09-28 11:27:25  2017-09-28  11:27:25   

      GPS Coordinaten     Lon_a      Lat_a     Lon_b      Lat_b  distance  \
0